In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
df = pd.read_csv('15stocks_price.csv')

df['date_gsheets'] = pd.to_datetime(df['date_gsheets'])
df.set_index('date_gsheets', inplace=True)

In [6]:
def getPortRisk(weights):
    returns_df = df.pct_change(1).dropna()
    num_stocks = len(returns_df.columns)
    vcv = returns_df.cov()
    
    var_p = np.dot(np.transpose(weights), np.dot(vcv, weights))
    sd_p = np.sqrt(var_p)
    sd_p_annual = sd_p * np.sqrt(250)
    
    return sd_p_annual

In [7]:
num_stocks = len(df.columns)
init_weights = [1 / num_stocks] * num_stocks

In [9]:
bounds = tuple((0,1) for i in range(num_stocks))

In [10]:
cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1})

In [11]:
results = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons)

In [12]:
results

     fun: 0.1059347936948761
     jac: array([0.10597393, 0.10585235, 0.10600164, 0.1058007 , 0.10597585,
       0.10618784, 0.10613558, 0.10582459, 0.10587699, 0.11916338,
       0.11288869, 0.10579355, 0.10934387, 0.1058938 , 0.10634915])
 message: 'Optimization terminated successfully'
    nfev: 176
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([4.79798174e-02, 2.80742498e-01, 7.53106181e-03, 1.29294763e-01,
       3.54257633e-02, 8.40458002e-02, 1.62830960e-01, 1.72868281e-01,
       2.00872883e-02, 0.00000000e+00, 1.62630326e-18, 3.55672017e-03,
       2.49366500e-18, 4.31645324e-02, 1.24725144e-02])

In [13]:
getPortRisk(init_weights)

0.13290599867193698

In [16]:
optimized_weights = pd.DataFrame(results['x'])
optimized_weights.index = df.columns
optimized_weights.rename(columns={optimized_weights.columns[0] : 'weights'}, inplace=True)

In [17]:
optimized_weights

,weights
AAPL,4.797982e-02
KO,2.807425e-01
NFLX,7.531062e-03
BRK.B,1.292948e-01
DIS,3.542576e-02
IBM,8.404580e-02
VZ,1.628310e-01
WMT,1.728683e-01
GE,2.008729e-02
TSLA,0.000000e+00


In [18]:
optimized_weights['weights_rounded'] = optimized_weights['weights'].apply(lambda x : round(x, 3))

In [19]:
optimized_weights

,weights,weights_rounded
AAPL,4.797982e-02,0.048
KO,2.807425e-01,0.281
NFLX,7.531062e-03,0.008
BRK.B,1.292948e-01,0.129
DIS,3.542576e-02,0.035
IBM,8.404580e-02,0.084
VZ,1.628310e-01,0.163
WMT,1.728683e-01,0.173
GE,2.008729e-02,0.020
TSLA,0.000000e+00,0.000


In [21]:
optimized_weights['weights_rounded'].sort_values(ascending=False).cumsum()

KO       0.281
WMT      0.454
VZ       0.617
BRK.B    0.746
IBM      0.830
AAPL     0.878
UN       0.921
DIS      0.956
GE       0.976
V        0.988
NFLX     0.996
AMZN     1.000
TSLA     1.000
MA       1.000
MSFT     1.000
Name: weights_rounded, dtype: float64